# Grids with Deep Learning

In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-0ubuntu1~19.04.1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/megan/Projects/h2oclass/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2enirtgy
  JVM stdout: /tmp/tmp2enirtgy/h2o_megan_started_from_python.out
  JVM stderr: /tmp/tmp2enirtgy/h2o_megan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_megan_avm2zu
H2O cluster total nodes:,1
H2O cluster free memory:,1.520 Gb
H2O cluster total cores:,3
H2O cluster allowed cores:,3
H2O cluster status:,"accepting new members, healthy"


In [3]:
# import the airlines data again
data = h2o.import_file('http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# split the data
train, valid, test = data.split_frame([0.8,0.1], seed=69)

In [5]:
# set up our x and y and exclude TailNum this time too
y = 'IsArrDelayed'
ignore_fields = ['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay',
                 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
                 'IsDepDelayed', 'IsArrDelayed', 'ActualElapsedTime',
                 'ArrTime', 'TailNum']
x_all = [i for i in train.names if i not in ignore_fields]

In [6]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [7]:
from h2o.grid.grid_search import H2OGridSearch

In [8]:
search_criteria = {'strategy': 'RandomDiscrete', 
                   'max_models': 12
}
hyper_params = {'seed': 77,
                'activation': ['RectifierWithDropout'],
                'l1': [0, 0.00001,0.000001,0.000003],
                'l2': [0, 0.00001,0.000001,0.000003],
                'input_dropout_ratio' : [0, 0.1, 0.2, 0.3],
                'hidden_dropout_ratios':[[0, 0], [0.2, 0.2], [0.4, 0.4], [0.6, 0.6]]
}

g = H2OGridSearch(
        model=H2ODeepLearningEstimator(epochs=40, hidden=[400, 400]),
        grid_id='grid_test',
        hyper_params=hyper_params,
        search_criteria=search_criteria
)

%time g.train(x_all, y, train, validation_frame = valid)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%
CPU times: user 3.59 s, sys: 216 ms, total: 3.8 s
Wall time: 22min 24s


In [10]:
g.summary()


Grid Summary:



Model Id,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
grid_test_model_7,1,299,Input,0.0,,,,,,,,,
grid_test_model_9,1,299,Input,0.0,,,,,,,,,
grid_test_model_11,1,299,Input,10.0,,,,,,,,,
grid_test_model_4,1,299,Input,10.0,,,,,,,,,
grid_test_model_12,1,299,Input,10.0,,,,,,,,,
grid_test_model_8,1,299,Input,10.0,,,,,,,,,
grid_test_model_10,1,299,Input,10.0,,,,,,,,,
grid_test_model_3,1,299,Input,10.0,,,,,,,,,
grid_test_model_2,1,299,Input,20.0,,,,,,,,,
grid_test_model_1,1,299,Input,30.0,,,,,,,,,
